
## **Part C** <font color="blue">(4 points)</font>
Apply SVM and random forests to a data set of your choice. Be sure to fit the models on a training set and to evaluate their performance on a test set. How accurate are the results compared to each other? Which of them yields the best performance?


In [1]:
# Importing necessary libraries

import numpy as np
import pandas as pd

#import statsmodels.api as sm
#from sklearn.metrics import mean_squared_error
#from math import sqrt

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn import svm #, linear_model

#from sklearn.model_selection import KFold, cross_val_score, train_test_split, GridSearchCV

#from sklearn.preprocessing import StandardScaler, PolynomialFeatures

from sklearn.ensemble import RandomForestClassifier

#from sklearn import metrics
#from sklearn.metrics import r2_score

#from sklearn.svm import SVR
#from sklearn.ensemble import RandomForestRegressor
#from sklearn.model_selection import GridSearchCV

In [2]:
# for each column, display column name & its distinct contents ONLY in case the column has less than 6 unique values
def get_columns_and_distinct_values(df,nunique_target=5):
  str_=""
  ret=[]
  for column in df:
      columnSeriesObj = df[column]
      nunique_this_clmn = columnSeriesObj.nunique()
      str_ = 'Column Name: ' + column + ', Column Type: ' + str(df[column].dtype)
      if nunique_this_clmn == 1:
          str_ = str_ + ', Column has one unique value only and can be dropped from the dataframe'
          ret.append(str_)
          ret.append("------------------------------------------------------------")
      elif nunique_this_clmn <= nunique_target:
        str_ = str_ + ', Column has ' + str(columnSeriesObj.nunique()) + ' unique value(s) : '
        for val in columnSeriesObj.unique():
          str_ = str_  + str(val) + ","
        str_ = str_[:len(str_)-1]
        ret.append(str_)
        ret.append("------------------------------------------------------------")
      else:
        str_ = str_ + ', Column has more than ' + str(nunique_target) + ' unique values'
      #ret.append(str_)
      #ret.append("==========================================================================")

  print("==========================================================================")
  columns_of_type_object = df.select_dtypes(include=['object']).columns
  print("*** Columns of type object are:", columns_of_type_object)
  print("==========================================================================")
  columns_of_a_single_unique_value = df.columns[df.nunique() == 1]
  print("*** Columns of a single unique value are:", columns_of_a_single_unique_value, " can be dropped")
  print("==========================================================================")
  return ret

##### Titanic Data Description
##### Passenger ID to identify the passenger, numerical feature (Passenger ID/Ticket Number).
##### Survived is our label, as we can see is a binary feature, 1 if survived and 0 otherwise.
##### Pclass is the Ticket class (1 = 1st (Upper), 2 = 2nd (Middle), 3 = 3rd (lower))
##### Age is the age in years
##### Sibsp is the number of siblings / spouses aboard the Titanic
##### Parch is the number of parents / children aboard the Titanic
##### Ticket is the ticket number
##### Fare is the Passenger fare
##### Cabin is the cabin number
##### Embarked means Port of Embarkation. C = Cherbourg, Q = Queenstown, S = Southampton

##### Titanic Train Data

In [3]:
# load train data from train.csv
titanic_train_df = pd.read_csv(r'train.csv')
titanic_train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


##### EDA Train Data

In [4]:
titanic_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
titanic_train_df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [6]:
titanic_train_df.shape

(891, 12)

In [7]:
# check for NULL values
if titanic_train_df.isnull().any().sum() > 0:
    print(titanic_train_df.isnull().any().sum())
else:
    print("Train data is good - There is no null values in any of the columns")

3


In [8]:
titanic_train_df.describe(include = 'all')

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,891.000000,891.000000,891.000000,891,891,714.000000,891.000000,891.000000,891,891.000000,204,889
unique,NaN,NaN,NaN,891,2,NaN,NaN,NaN,681,NaN,147,3
top,NaN,NaN,NaN,"Braund, Mr. Owen Harris",male,NaN,NaN,NaN,347082,NaN,B96 B98,S
freq,NaN,NaN,NaN,1,577,NaN,NaN,NaN,7,NaN,4,644
mean,446.000000,0.383838,2.308642,NaN,NaN,29.699118,0.523008,0.381594,NaN,32.204208,NaN,NaN
std,257.353842,0.486592,0.836071,NaN,NaN,14.526497,1.102743,0.806057,NaN,49.693429,NaN,NaN
min,1.000000,0.000000,1.000000,NaN,NaN,0.420000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,223.500000,0.000000,2.000000,NaN,NaN,20.125000,0.000000,0.000000,NaN,7.910400,NaN,NaN
50%,446.000000,0.000000,3.000000,NaN,NaN,28.000000,0.000000,0.000000,NaN,14.454200,NaN,NaN
75%,668.500000,1.000000,3.000000,NaN,NaN,38.000000,1.000000,0.000000,NaN,31.000000,NaN,NaN


In [9]:
print ("titanic_train_df:")
#print("null values:\n", titanic_train_df.isnull().sum())
print("duplicated records", titanic_train_df.duplicated().sum())

titanic_train_df:
duplicated records 0


##### Handle Nan values

###### Embarked Feature / Two Records with Nan values - Drop

In [10]:
# Two records in titanic_train_df with Nan in Embarked feature  - will be dropped from the dataframe
titanic_train_df[titanic_train_df['Embarked'].isnull()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
61,62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN
829,830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN


In [11]:
titanic_train_df.dropna(subset=['Embarked'],inplace=True)
titanic_train_df[titanic_train_df['Embarked'].isnull()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked


###### Cabin Feature / replace missing values in titanic_train_df['Cabin'] with letter 'U' for unknown

In [12]:
print("null values in the Cabin Feature:\n", titanic_train_df.isnull().sum()['Cabin'], "Records")

null values in the Cabin Feature:
 687 Records


In [13]:
titanic_train_df['Cabin'].unique()

array([nan, 'C85', 'C123', 'E46', 'G6', 'C103', 'D56', 'A6',
       'C23 C25 C27', 'B78', 'D33', 'B30', 'C52', 'C83', 'F33', 'F G73',
       'E31', 'A5', 'D10 D12', 'D26', 'C110', 'B58 B60', 'E101', 'F E69',
       'D47', 'B86', 'F2', 'C2', 'E33', 'B19', 'A7', 'C49', 'F4', 'A32',
       'B4', 'B80', 'A31', 'D36', 'D15', 'C93', 'C78', 'D35', 'C87',
       'B77', 'E67', 'B94', 'C125', 'C99', 'C118', 'D7', 'A19', 'B49',
       'D', 'C22 C26', 'C106', 'C65', 'E36', 'C54', 'B57 B59 B63 B66',
       'C7', 'E34', 'C32', 'B18', 'C124', 'C91', 'E40', 'T', 'C128',
       'D37', 'B35', 'E50', 'C82', 'B96 B98', 'E10', 'E44', 'A34', 'C104',
       'C111', 'C92', 'E38', 'D21', 'E12', 'E63', 'A14', 'B37', 'C30',
       'D20', 'B79', 'E25', 'D46', 'B73', 'C95', 'B38', 'B39', 'B22',
       'C86', 'C70', 'A16', 'C101', 'C68', 'A10', 'E68', 'B41', 'A20',
       'D19', 'D50', 'D9', 'A23', 'B50', 'A26', 'D48', 'E58', 'C126',
       'B71', 'B51 B53 B55', 'D49', 'B5', 'B20', 'F G63', 'C62 C64',
       'E24',

In [14]:
# keep first letter only in the Cabin
titanic_train_df['Cabin'] = titanic_train_df['Cabin'].str[0]
titanic_train_df['Cabin'].unique()

array([nan, 'C', 'E', 'G', 'D', 'A', 'B', 'F', 'T'], dtype=object)

In [15]:
# replace missing values in titanic_train_df['Cabin'] with letter 'U' for unknown
titanic_train_df['Cabin'].fillna('U', inplace=True)

In [16]:
titanic_train_df['Cabin'].unique()

array(['U', 'C', 'E', 'G', 'D', 'A', 'B', 'F', 'T'], dtype=object)

###### Age Feature / replace missing values in titanic_train_df['Age'] with the avergae of age

In [18]:
print("null values in the Age Feature:\n", titanic_train_df.isnull().sum()['Age'], "Records")

null values in the Age Feature:
 177 Records


In [19]:
# Fill missing/Nan values in Age with average age
titanic_train_df['Age'].fillna(titanic_train_df['Age'].mean(), inplace=True)  # 29.642093
print("null values in the Age Feature:\n", titanic_train_df.isnull().sum()['Age'], "Records")
#titanic_train_df['Age'].unique()


null values in the Age Feature:
 0 Records


In [46]:
# Check again after fixing missing records
titanic_train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 889 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  889 non-null    int64  
 1   Survived     889 non-null    int64  
 2   Pclass       889 non-null    int64  
 3   Name         889 non-null    object 
 4   Sex          889 non-null    object 
 5   Age          889 non-null    float64
 6   SibSp        889 non-null    int64  
 7   Parch        889 non-null    int64  
 8   Ticket       889 non-null    object 
 9   Fare         889 non-null    float64
 10  Cabin        889 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 90.3+ KB


In [21]:
titanic_train_df.describe(include = 'all')

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,889.000000,889.000000,889.000000,889,889,889.000000,889.000000,889.000000,889,889.000000,889,889
unique,NaN,NaN,NaN,889,2,NaN,NaN,NaN,680,NaN,9,3
top,NaN,NaN,NaN,"Braund, Mr. Owen Harris",male,NaN,NaN,NaN,347082,NaN,U,S
freq,NaN,NaN,NaN,1,577,NaN,NaN,NaN,7,NaN,687,644
mean,446.000000,0.382452,2.311586,NaN,NaN,29.642093,0.524184,0.382452,NaN,32.096681,NaN,NaN
std,256.998173,0.486260,0.834700,NaN,NaN,12.968346,1.103705,0.806761,NaN,49.697504,NaN,NaN
min,1.000000,0.000000,1.000000,NaN,NaN,0.420000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,224.000000,0.000000,2.000000,NaN,NaN,22.000000,0.000000,0.000000,NaN,7.895800,NaN,NaN
50%,446.000000,0.000000,3.000000,NaN,NaN,29.642093,0.000000,0.000000,NaN,14.454200,NaN,NaN
75%,668.000000,1.000000,3.000000,NaN,NaN,35.000000,1.000000,0.000000,NaN,31.000000,NaN,NaN


In [22]:
titanic_train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,U,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,U,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,U,S


In [23]:
ret=get_columns_and_distinct_values(titanic_train_df,10)
for str_ in ret:
  print(str_)

*** Columns of type object are: Index(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'], dtype='object')
*** Columns of a single unique value are: Index([], dtype='object')  can be dropped
Column Name: Survived, Column Type: int64, Column has 2 unique value(s) : 0,1
------------------------------------------------------------
Column Name: Pclass, Column Type: int64, Column has 3 unique value(s) : 3,1,2
------------------------------------------------------------
Column Name: Sex, Column Type: object, Column has 2 unique value(s) : male,female
------------------------------------------------------------
Column Name: SibSp, Column Type: int64, Column has 7 unique value(s) : 1,0,3,4,2,5,8
------------------------------------------------------------
Column Name: Parch, Column Type: int64, Column has 7 unique value(s) : 0,1,2,5,3,4,6
------------------------------------------------------------
Column Name: Cabin, Column Type: object, Column has 9 unique value(s) : U,C,E,G,D,A,B,F,T
-----------

##### Titanic Test Data

In [24]:
# load data
titanic_test_df = pd.read_csv(r'test.csv')
titanic_test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


##### EDA Test Data

In [25]:
titanic_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [26]:
titanic_test_df.columns

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [27]:
titanic_test_df.shape

(418, 11)

In [28]:
titanic_test_df.columns

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [29]:
titanic_test_df.describe(include = 'all')

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,418.000000,418.000000,418,418,332.000000,418.000000,418.000000,418,417.000000,91,418
unique,NaN,NaN,418,2,NaN,NaN,NaN,363,NaN,76,3
top,NaN,NaN,"Kelly, Mr. James",male,NaN,NaN,NaN,PC 17608,NaN,B57 B59 B63 B66,S
freq,NaN,NaN,1,266,NaN,NaN,NaN,5,NaN,3,270
mean,1100.500000,2.265550,NaN,NaN,30.272590,0.447368,0.392344,NaN,35.627188,NaN,NaN
std,120.810458,0.841838,NaN,NaN,14.181209,0.896760,0.981429,NaN,55.907576,NaN,NaN
min,892.000000,1.000000,NaN,NaN,0.170000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,996.250000,1.000000,NaN,NaN,21.000000,0.000000,0.000000,NaN,7.895800,NaN,NaN
50%,1100.500000,3.000000,NaN,NaN,27.000000,0.000000,0.000000,NaN,14.454200,NaN,NaN
75%,1204.750000,3.000000,NaN,NaN,39.000000,1.000000,0.000000,NaN,31.500000,NaN,NaN


In [30]:
if titanic_test_df.isnull().any().sum() > 0:
    print(titanic_test_df.isnull().any().sum())
else:
    print("Test data is good - There is no null values in any of the columns")

3


In [31]:
titanic_test_df.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

###### Fare Feature / One Record with Nan values - replace with average Fare

In [32]:
# One record in titanic_test_df with Nan in Fare feature  - will be dropped from the dataframe
titanic_test_df[titanic_test_df['Fare'].isnull()]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
152,1044,3,"Storey, Mr. Thomas",male,60.5,0,0,3701,NaN,NaN,S


In [33]:
# Fill missing/Nan values in Fare with average Fare
titanic_test_df['Fare'].fillna(titanic_test_df['Fare'].mean(), inplace=True)  # 35.627188
print("null values in the Age Feature:\n", titanic_test_df.isnull().sum()['Fare'], "Records")

null values in the Age Feature:
 0 Records


###### Cabin Feature / replace missing values in titanic_test_df['Cabin'] with letter 'U' for unknown

In [34]:
print("null values in the Cabin Feature:\n", titanic_test_df.isnull().sum()['Cabin'], "Records")

null values in the Cabin Feature:
 327 Records


In [35]:
titanic_test_df['Cabin'].unique()

array([nan, 'B45', 'E31', 'B57 B59 B63 B66', 'B36', 'A21', 'C78', 'D34',
       'D19', 'A9', 'D15', 'C31', 'C23 C25 C27', 'F G63', 'B61', 'C53',
       'D43', 'C130', 'C132', 'C101', 'C55 C57', 'B71', 'C46', 'C116',
       'F', 'A29', 'G6', 'C6', 'C28', 'C51', 'E46', 'C54', 'C97', 'D22',
       'B10', 'F4', 'E45', 'E52', 'D30', 'B58 B60', 'E34', 'C62 C64',
       'A11', 'B11', 'C80', 'F33', 'C85', 'D37', 'C86', 'D21', 'C89',
       'F E46', 'A34', 'D', 'B26', 'C22 C26', 'B69', 'C32', 'B78',
       'F E57', 'F2', 'A18', 'C106', 'B51 B53 B55', 'D10 D12', 'E60',
       'E50', 'E39 E41', 'B52 B54 B56', 'C39', 'B24', 'D28', 'B41', 'C7',
       'D40', 'D38', 'C105'], dtype=object)

In [36]:
# keep first letter only in the Cabin
titanic_test_df['Cabin'] = titanic_test_df['Cabin'].str[0]
titanic_test_df['Cabin'].unique()

array([nan, 'B', 'E', 'A', 'C', 'D', 'F', 'G'], dtype=object)

In [37]:
# replace missing values in titanic_test_df['Cabin'] with letter 'U' for unknown
titanic_test_df['Cabin'].fillna('U', inplace=True)

In [38]:
titanic_test_df['Cabin'].unique()

array(['U', 'B', 'E', 'A', 'C', 'D', 'F', 'G'], dtype=object)

###### Age Feature / replace missing values in titanic_test_df['Age'] with the avergae of age

In [40]:
print("null values in the Age Feature:\n", titanic_test_df.isnull().sum()['Age'], "Records")

null values in the Age Feature:
 86 Records


In [41]:
# Fill missing/Nan values in Age with average age
titanic_test_df['Age'].fillna(titanic_test_df['Age'].mean(), inplace=True)  # 30.272590
print("null values in the Age Feature:\n", titanic_test_df.isnull().sum()['Age'], "Records")
#titanic_test_df['Age'].unique()


null values in the Age Feature:
 0 Records


In [42]:
# Check again after fixing missing records
titanic_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          418 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         418 non-null    float64
 9   Cabin        418 non-null    object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [43]:
titanic_test_df.describe(include = 'all')

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,418.000000,418.000000,418,418,418.000000,418.000000,418.000000,418,418.000000,418,418
unique,NaN,NaN,418,2,NaN,NaN,NaN,363,NaN,8,3
top,NaN,NaN,"Kelly, Mr. James",male,NaN,NaN,NaN,PC 17608,NaN,U,S
freq,NaN,NaN,1,266,NaN,NaN,NaN,5,NaN,327,270
mean,1100.500000,2.265550,NaN,NaN,30.272590,0.447368,0.392344,NaN,35.627188,NaN,NaN
std,120.810458,0.841838,NaN,NaN,12.634534,0.896760,0.981429,NaN,55.840500,NaN,NaN
min,892.000000,1.000000,NaN,NaN,0.170000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,996.250000,1.000000,NaN,NaN,23.000000,0.000000,0.000000,NaN,7.895800,NaN,NaN
50%,1100.500000,3.000000,NaN,NaN,30.272590,0.000000,0.000000,NaN,14.454200,NaN,NaN
75%,1204.750000,3.000000,NaN,NaN,35.750000,1.000000,0.000000,NaN,31.500000,NaN,NaN


In [44]:
titanic_test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,U,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,U,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,U,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,U,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,U,S


In [45]:
ret=get_columns_and_distinct_values(titanic_test_df,10)
for str_ in ret:
  print(str_)

*** Columns of type object are: Index(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'], dtype='object')
*** Columns of a single unique value are: Index([], dtype='object')  can be dropped
Column Name: Pclass, Column Type: int64, Column has 3 unique value(s) : 3,2,1
------------------------------------------------------------
Column Name: Sex, Column Type: object, Column has 2 unique value(s) : male,female
------------------------------------------------------------
Column Name: SibSp, Column Type: int64, Column has 7 unique value(s) : 0,1,2,3,4,5,8
------------------------------------------------------------
Column Name: Parch, Column Type: int64, Column has 8 unique value(s) : 0,1,3,2,4,6,5,9
------------------------------------------------------------
Column Name: Cabin, Column Type: object, Column has 8 unique value(s) : U,B,E,A,C,D,F,G
------------------------------------------------------------
Column Name: Embarked, Column Type: object, Column has 3 unique value(s) : Q,S,C
--------